In [5]:
import tkinter as tk
import pymysql
from gtts import gTTS
from playsound import playsound


def generate_and_add_audio():
    text = text_input.get("1.0", "end-1c")

    filename = filename_entry.get()

    generate_audio(text, filename)

    listbox.insert(tk.END, filename)

    conn = pymysql.connect(host='localhost', user='root', password='', db='audio_files')
    cur = conn.cursor()
    cur.execute("INSERT INTO audio_files (filename) VALUES (%s)", filename)
    conn.commit()
    cur.close()
    conn.close()

def fetch_filenames():

    listbox.delete(0, tk.END)

    conn = pymysql.connect(host='localhost', user='root', password='', db='audio_files')
    cur = conn.cursor()
    cur.execute("SELECT filename FROM audio_files")
    filenames = cur.fetchall()
    cur.close()
    conn.close()

    for filename in filenames:
        listbox.insert(tk.END, filename[0])

def exit_application():
    root.destroy()

def generate_audio(text, filename):

    tts = gTTS(text=text, lang='en')

    modified_filename = filename + '.mp3'

    tts.save(modified_filename)

    return modified_filename

def play_audio():
    selection = listbox.curselection()
    if selection:
        filename = listbox.get(selection[0])
        modified_filename = filename + '.mp3'
        playsound(modified_filename)

root = tk.Tk()
root.title('Text to Speech Converter™ by Akshay khandare')
root.geometry('800x500')
root.configure(bg='#b3e6ff')

title_label = tk.Label(root, text='Text to Speech Converter', font=('Arial', 24), bg='black', fg='orange')
title_label.place(relx=0.5, rely=0.05, anchor='center')

left_frame = tk.Frame(root, bg='#f2f2f2', padx=20, pady=20)
left_frame.place(relx=0.1, rely=0.15, relwidth=0.4, relheight=0.73)

right_frame = tk.Frame(root, bg='#f2f2f2', padx=20, pady=20)
right_frame.place(relx=0.55, rely=0.15, relwidth=0.3, relheight=0.73)

filename_label = tk.Label(left_frame, text="Enter the Text", fg='black', font=('Arial', 16))
filename_label.pack(pady=5)

def validate_text_input(text):
    if text.isalpha() or text.isspace():
        return True
    else:
        return False

validate_input = (root.register(validate_text_input), '%S')

text_input = tk.Text(left_frame, width=30, height=6, font=('Arial', 12))
text_input.pack(pady=8)

filename_label = tk.Label(left_frame, text="Enter the Filename",fg='black', font=('Arial', 14))
filename_label.pack(pady=8)

filename_entry = tk.Entry(left_frame, width=30, font=('Arial', 14))
filename_entry.pack(pady=8)

generate_button = tk.Button(left_frame, text='Generate Audio', command=generate_and_add_audio, bg='black', fg='#f2f2f2', font=('Arial', 12))
generate_button.pack(pady=20)

listbox = tk.Listbox(right_frame, width=30, height=15, font=('Arial', 16))
listbox.pack()

fetch_filenames()

listbox.bind('<Double-Button-1>', lambda event: play_audio())

exit_button = tk.Button(root, text='Exit', command=exit_application, bg='#ff4d4d', fg='white', font=('Arial', 14))
exit_button.place(relx=0.45, rely=0.9, relwidth=0.1, relheight=0.08)

root.mainloop()